# PyTorch Deep Learning to Federated Learning Conversion


One common question frequently heard from data scientists is "how do I write federated learning code? If I already have training code for deep learning, how do I write federated learning training code for the same problem?"

In this section, we will look at the classification training code we ran earlier and see how to convert the existing PyTorch training script to federated learning client training code.


## Orginal Deep learning Training Script

In [ ]:
%cd code

In [ ]:
! python3 src/client_origin.py

## Convert the Deep learning Training Script

Now let's convert it to federated learning training code with NVIDIA FLARE's Client API


**Step-1** import

```
import nvflare.client as flare

```

**Step-2** init

we call 

```python
flare.init()
```

Once the flare is initialized, we will recieve some system metadata for example

```python

  sys_info = flare.system_info()
  client_name = sys_info["site_name"]

```
We can get current client's "identity". 

Next we need to extend the training beyond local iterations. Imagine the Federated Learning is like the following for-loop:

```python

rounds = 5
for current_round in ranage (rounds):
     
    <at each site local_training()>

```

Therefore we need to additional loop for the Federated Learning training. This can be expressed 

**Step 3** global round loop 

    while flare.is_running():
        <local training>


For each round: we need to receive and evaluate the global model. 


**Step-4** Receive global model

```python

        input_model = flare.receive()
        round=input_model.current_round

        # update model based on global model
        model.load_state_dict(input_model.params)
```

**Step-5** Evaluate Global Model

Since the local model is being updated with global model, the training procedure calculates the loss which evaluates the model


**Step-6** Send the local trained model back to aggregator

We take the newly trained local model parameters as well as metadata, send it back to aggregator.

```python

        output_model = flare.FLModel( params=model.cpu().state_dict(), meta={"NUM_STEPS_CURRENT_ROUND": steps},)

        flare.send(output_model)
```

With above steps, just a few lines of code changes, no code structural changes, we converted the PyTorch deep learning code to federated learning with NVIDIA FLARE.

The complete code can be found at client.py

In [ ]:
!cat code/src/client_v1.py

Now, we converted the client PyTorch training script to federated learning code. Let's look further to handle multi-task client code.


## Multi-Task Client Scripts

So far, the client only handles training, regardless of what tasks the server issues to the clients. What if there are many tasks? Client should take different actions based on the different tasks. Also, in the previous version, we did not evaluate the global model. We are going to handle all these in this section.


In Flare's Client API, by default, we will issue three different tasks: "train", "evaluate" and "submit_model"

These three tasks can be checked by 

```python

flare.is_train()

flare.is_evaluate()

flare.is_submit_model()

```

So we need to modify our existing training code to have both training and evaluation logics.

### Training logics changes

Besides the training logics we have seen before. We also need to evaluate and obtain the accuracy of the trainiing. 
here we perform two evaluates 

evaluate the local model: 

```python
            # (5.2) evaluation on local trained model to save best model
            local_accuracy = evaluate(net.state_dict())


```

evaluate the global model received:

```python

     # (5.3) evaluate on received model for model selection
            accuracy = evaluate(input_model.params)
```

Then add the global model accuracy into the metrics parameter of the FLModel before send it back to server. 

```python

 output_model = flare.FLModel(
                params=net.cpu().state_dict(),
                metrics={"accuracy": accuracy},
                meta={"NUM_STEPS_CURRENT_ROUND": steps},
            )
```


The newly added training logics is like this. 

>Note: the evaluate() function will discussed next


```python
 

            # (5.2) evaluation on local trained model to save best model
            local_accuracy = evaluate(net.state_dict())
            print(f"({client_id}) Evaluating local trained model. Accuracy on the 10000 test images: {local_accuracy}")
            if local_accuracy > best_accuracy:
                best_accuracy = local_accuracy
                torch.save(net.state_dict(), model_path)

            # (5.3) evaluate on received model for model selection
            accuracy = evaluate(input_model.params)
            print(
                f"({client_id}) Evaluating received model for model selection. Accuracy on the 10000 test images: {accuracy}"
            )

            # (5.4) construct trained FL model
            output_model = flare.FLModel(
                params=net.cpu().state_dict(),
                metrics={"accuracy": accuracy},
                meta={"NUM_STEPS_CURRENT_ROUND": steps},
            )

            # (5.5) send model back to NVFlare
            flare.send(output_model)

```

### Evaluate functions

The evaluate() functions requires test data, it is a nested inner evaluation that can directly use the testloader. 
The return value is accuracy percentage. 


```python

    # wraps evaluation logic into a method to re-use for
    #       evaluation on both trained and received model
    def evaluate(input_weights):
        net = Net()
        net.load_state_dict(input_weights)
        # (optional) use GPU to speed things up
        net.to(DEVICE)

        correct = 0
        total = 0
        # since we're not training, we don't need to calculate the gradients for our outputs
        with torch.no_grad():
            for data in testloader:

                # (optional) use GPU to speed things up
                images, labels = data[0].to(DEVICE), data[1].to(DEVICE)

                # calculate outputs by running images through the network
                outputs = net(images)

                # the class with the highest energy is what we choose as prediction

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        return 100 * correct // total

```


The overall logics becomes

```python

if flare.is_training(): 
    traing and evaluate metrics
    send model and merics back

elif flare.is_evaluate():
    # evaluate only, this can be used for cross-site evaluation
    evaluate()
    send the model and metrics back 

elif flare.is_submit_model()
    
    # expecting client submit best model 
    load and set best model 

```

Please take a look at the [client.py](./code/src/client.py)


In [ ]:
!cat code/src/client.py

Now, we know how to convert an existing Deep Learning code to Federated Learning training script. We can now explore how to customize the training logics. 

Please checkout [server side customization](../01.3_server_side_customization/customize_server_logics.ipynb)

